In [2]:
# -*- coding: utf-8 -*-
# 作者: xcl
# 时间: 2019/8/1 21:51
import pandas as pd
import keras
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, core, add
from keras.models import Model
import os
import numpy as np

Using TensorFlow backend.


In [22]:
# 读取
input_path = 'D:\\全2018_日出足够.xlsx'
data_all = pd.read_excel(input_path, index_col='日期')
data_all = data_all.dropna()


In [99]:
print(data_all.shape)

(29753, 111)


In [100]:
data_ts_df = data_all[['tm_mon', 'tm_mday', 'tm_wday', 'tm_yday', 'tm_week','id']]
data_to_std = data_all.drop(['tm_mon', 'tm_mday', 'tm_wday', 'tm_yday', 'tm_week','id'], axis=1)

In [149]:
# 标准化
from sklearn.preprocessing import MinMaxScaler
data_to_std2= MinMaxScaler().fit_transform(data_to_std)
data_to_std2 = pd.DataFrame(data_to_std2)
data_to_std2 = data_to_std2.set_index(data_to_std.index)
data_to_std2.columns = data_to_std.columns
print(data_to_std2.shape)
print(data_ts_df.shape)

(29753, 105)
(29753, 6)


In [150]:
print(data_to_std2.shape)
print(data_ts_df.shape)
print(data_ts_df.columns)
print(data_ts_df.index)

(29753, 105)
(29753, 6)
Index(['tm_mon', 'tm_mday', 'tm_wday', 'tm_yday', 'tm_week', 'id'], dtype='object')
Index(['2018-08-15', '2018-08-16', '2018-08-17', '2018-08-18', '2018-08-19',
       '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24',
       ...
       '2018-05-06', '2018-05-07', '2018-05-08', '2018-05-09', '2018-05-10',
       '2018-05-11', '2018-05-12', '2018-05-13', '2018-05-14', '2018-05-15'],
      dtype='object', name='日期', length=29753)


In [151]:
data_out = pd.concat([data_ts_df, data_to_std2],join='outer', axis=1)
#data_out2.to_csv('test.csv')

In [273]:
data_test = data_out[(data_out['tm_mon']==1)|
                     (data_out['tm_mon']==4)|
                    (data_out['tm_mon']==7)|
                    (data_out['tm_mon']==10)]
data_train = data_out[(data_out['tm_mon']==3)|
                     (data_out['tm_mon']==2)|
                    (data_out['tm_mon']==6)|
                    (data_out['tm_mon']==5)|
                     (data_out['tm_mon']==9)|
                    (data_out['tm_mon']==8)|
                    (data_out['tm_mon']==12)|
                     (data_out['tm_mon']==11)]

In [274]:
# AOD
data_aod_test = data_test[['AOD_0']]
data_aods_test = data_test[[
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

data_aod_train = data_train[['AOD_0']]
data_aods_train = data_train[[
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

In [275]:
# 气象
data_sky_test = data_test[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'pressure',
                     'sunTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]

data_sky_train = data_train[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'pressure',
                     'sunTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]


In [276]:
# 时空特征
data_ts_test = data_test[['tm_mon','tm_wday','id']]
print(data_ts_test.shape)
data_ts_train = data_train[['tm_mon','tm_wday','id']]

(9654, 3)


In [277]:
# 时滞
data_t1_test = data_test[['AOD_0_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]

data_t1_train = data_train[['AOD_0_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]

In [278]:
# NDVI
data_ndvi_test = data_test[['NDVI_0','AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]
data_ndvi_train = data_train[['NDVI_0','AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

In [279]:
# 污染物
data_pm_test = data_test[['PM25']]
data_pm_train = data_train[['PM25']]

In [384]:
# 第一个模块 当日天气模块
#
# 输入层
Meteorology_input = Input(shape=(len(data_sky_test.columns),), name="Meteorology_input")
AODs_input = Input(shape=(len(data_aod_test.columns),), name="AODs_input")
# 因素融合层
meteorology_concat = concatenate([Meteorology_input, AODs_input])
# 全连接层
meteorology_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_1")(meteorology_concat)
# 残差层
meteorology_residual_connection = Dense(
    16,
    activation=keras.layers.LeakyReLU(alpha=0.3),
    name="ResidualConnectionMA")(meteorology_x)
meteorology_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_RC")(meteorology_residual_connection)
meteorology_residual_connection = add(
    [meteorology_x, meteorology_residual_connection], name="ResidualConnectionMA_Add")
# 全连接层
meteorology_x = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_2")(meteorology_residual_connection)
# Dropout
meteorology_y = core.Dropout(
    rate=0.5, name="Meteorology_Module")(meteorology_x)
# 模型层
model_meteorology = Model(
    inputs=[
        Meteorology_input,
        AODs_input],
    outputs=meteorology_y)

In [385]:
# 第二个模块 时滞天气模块

Weather_input = Input(shape=(len(data_t1_test.columns),), name="Weather_input")  # 只添加新的因素即可
# AODs_input = Input(shape=(17,), name="AODs_input")
# 因素融合层
weather_concat = concatenate([Weather_input, AODs_input])
# 全连接层
weather_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWA_1")(weather_concat)
# 残差层
weather_residual_connection = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionWA")(weather_x)
weather_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWAForRC")(weather_residual_connection)
weather_residual_connection = add(
    [weather_x, weather_residual_connection], name="ResidualConnectionWA_Add")
# 全连接层
weather_x = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWA_2")(weather_residual_connection)
# Dropout
# use `rate` instead of `keep_prob`. r=1-p.
weather_y = core.Dropout(rate=0.5, name="Weather_Module")(weather_x)

# 模型层
model_weather = Model(inputs=[Weather_input, AODs_input], outputs=weather_y)



In [386]:
# 第三个模块 其他空气污染物模块 更改为ndvi
# 输入层
OtherPollution_input = Input(shape=(len(data_ndvi_test.columns),))
# 因素融合层
otherpollution_concat = concatenate([OtherPollution_input, AODs_input])
# 全连接层
otherpollution_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_1")(otherpollution_concat)
# 残差层
otherpollution_residual_connection = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionOPA")(otherpollution_x)
otherpollution_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_RC")(otherpollution_residual_connection)
otherpollution_residual_connection = add(
    [otherpollution_x, otherpollution_residual_connection], name="ResidualConnectionOPA_Add")
# 全连接层
otherpollution_x = Dense(
    6,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_2")(otherpollution_residual_connection)
# Dropout
otherpollution_y = core.Dropout(
    rate=0.5, name="OtherPollution_Module")(otherpollution_x)
# 模型层
model_otherpollution = Model(
    inputs=[
        OtherPollution_input,
        AODs_input],
    outputs=otherpollution_y)


In [398]:
###### 第四个模块 时空元属性模块
# 输入层
TimeStation_input = Input(shape=(len(data_ts_train.columns),), name="TimeStation_input")

# 因素融合层
timestation_concat = concatenate([TimeStation_input, AODs_input])
# 全连接层
timestation_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOTSA_1")(timestation_concat)
# 残差层
timestation_residual_connection = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionTSA")(timestation_x)
timestation_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_RC")(timestation_residual_connection)
timestation_residual_connection = add(
    [timestation_x, timestation_residual_connection], name="ResidualConnectionTSA_Add")
# 全连接层
timestation_x = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_2")(timestation_residual_connection)
timestation_x = Dense(
    4,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_222")(timestation_x)
# Dropout
timestation_y = core.Dropout(
    rate=0.5, name="TimeStation_Module")(timestation_x)
# 模型层
model_timestation = Model(
    inputs=[
        TimeStation_input,
        AODs_input],
    outputs=timestation_y)

In [399]:
# 第五个模块 整体影响模块
# 因素融合层
allin_concat = concatenate([Meteorology_input,
                            Weather_input,
                            OtherPollution_input,
                            TimeStation_input,
                            AODs_input])
# 全连接层
allin_x = Dense(
    48,
    activation=keras.layers.LeakyReLU(alpha=0.5),
    name="FullConnectionAIA_1")(allin_concat)
# 残差层
allin_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.5),
    name="ResidualConnectionAIA")(allin_x)
allin_residual_connection = Dense(
    48,
    activation=keras.layers.LeakyReLU(alpha=0.5),
    name="FullConnectionAIA_RC")(allin_residual_connection)
allin_residual_connection = add(
    [allin_x, allin_residual_connection], name="ResidualConnectionAIA_Add")
# 全连接层
allin_x = Dense(18, activation=keras.layers.LeakyReLU(alpha=0.5),
                name="FullConnectionAIA_2")(allin_residual_connection)
# Dropout
allin_y = core.Dropout(rate=0.6, name="AllIn_Module")(allin_x)
# 模型层
model_allin = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=allin_y)


W0902 17:54:04.541798 18136 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [406]:
# 最后的融合
# 捕捉的影响的融合层
res_concat = concatenate([model_meteorology.output,
                          model_weather.output,
                          model_otherpollution.output,
                          model_timestation.output,
                          model_allin.output])
# 全连接层
res_x = Dense(
    64,
    name="ResFullConnectionResModelForLast")(res_concat)
# 残差连接层
res_residual_connection = Dense(
    48,
    name="ResidualConnectionLast")(res_x)
res_residual_connection = Dense(
    64,
    name="FullConnectionLast_RC")(res_residual_connection)
res_residual_connection = add(
    [res_x, res_residual_connection], name="ResidualConnectionLast_Add")
# 全连接层
res_x = Dense(24, activation=keras.layers.ELU(alpha=0.8),
              name="FullConnectionLast_2")(res_residual_connection)
res_x2 = Dense(8, activation=keras.layers.ELU(alpha=0.8),
              name="FullConnectionLast_2x")(res_x)
# Dropout
res_y = core.Dropout(rate=0.1, name="Res_Module")(res_x2)
# sigmoid 最终融合结果
res_outcome = Dense(
    1,
    activation=keras.layers.LeakyReLU(alpha=0.25),
    kernel_regularizer=keras.regularizers.l2(0.1),
    name="sigmoid_FC")(res_y)
# 编译最终模型
model_last = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=res_outcome)
model_last.compile(loss="mse", optimizer='AdaDelta', metrics=["accuracy"])


In [ ]:

# print(model_last.summary())
model_last.fit([data_sky_train,data_t1_train,data_ndvi_train,data_ts_train,data_aod_train],data_pm_train,epochs = 200000, 
               batch_size =1536)


Epoch 1/200000
20099/20099 [==============================] - 6s 290us/step - loss: 0.4757 - acc: 9.9507e-05
Epoch 2/200000
20099/20099 [==============================] - 0s 22us/step - loss: 0.2479 - acc: 1.4926e-04
Epoch 3/200000
20099/20099 [==============================] - 1s 28us/step - loss: 0.2280 - acc: 1.4926e-04
Epoch 4/200000
20099/20099 [==============================] - 1s 28us/step - loss: 0.2144 - acc: 9.9507e-05
Epoch 5/200000
20099/20099 [==============================] - 0s 23us/step - loss: 0.2037 - acc: 1.4926e-04
Epoch 6/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.1944 - acc: 9.9507e-05
Epoch 7/200000
20099/20099 [==============================] - 0s 22us/step - loss: 0.1853 - acc: 9.9507e-05
Epoch 8/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.1779 - acc: 9.9507e-05
Epoch 9/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.1700 - acc: 1.4926e-04
Epoch 10/200000
20099/20099

20099/20099 [==============================] - 0s 20us/step - loss: 0.0042 - acc: 9.9507e-05
Epoch 76/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0041 - acc: 1.4926e-04
Epoch 77/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0042 - acc: 9.9507e-05
Epoch 78/200000
20099/20099 [==============================] - 0s 24us/step - loss: 0.0041 - acc: 9.9507e-05
Epoch 79/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0043 - acc: 1.4926e-04
Epoch 80/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0041 - acc: 9.9507e-05
Epoch 81/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0042 - acc: 1.4926e-04
Epoch 82/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0040 - acc: 9.9507e-05
Epoch 83/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0042 - acc: 9.9507e-05
Epoch 84/200000
20099/20099 [======

20099/20099 [==============================] - 0s 20us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 150/200000
20099/20099 [==============================] - 0s 23us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 151/200000
20099/20099 [==============================] - 0s 22us/step - loss: 0.0039 - acc: 9.9507e-05
Epoch 152/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0040 - acc: 9.9507e-05
Epoch 153/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0039 - acc: 9.9507e-05
Epoch 154/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0040 - acc: 9.9507e-05
Epoch 155/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0039 - acc: 9.9507e-05
Epoch 156/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0040 - acc: 9.9507e-05
Epoch 157/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0039 - acc: 9.9507e-05
Epoch 158/200000
20099/2009

20099/20099 [==============================] - 0s 22us/step - loss: 0.0036 - acc: 1.4926e-04
Epoch 224/200000
20099/20099 [==============================] - 0s 22us/step - loss: 0.0036 - acc: 9.9507e-05
Epoch 225/200000
20099/20099 [==============================] - 0s 23us/step - loss: 0.0036 - acc: 9.9507e-05
Epoch 226/200000
20099/20099 [==============================] - 0s 24us/step - loss: 0.0037 - acc: 9.9507e-05
Epoch 227/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0036 - acc: 1.4926e-04
Epoch 228/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0037 - acc: 9.9507e-05
Epoch 229/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0036 - acc: 1.4926e-04
Epoch 230/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0037 - acc: 1.4926e-04
Epoch 231/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0037 - acc: 9.9507e-05
Epoch 232/200000
20099/2009

20099/20099 [==============================] - 0s 21us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 298/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0035 - acc: 9.9507e-05: 0s - loss: 0.0035 - acc: 1.0016e-0
Epoch 299/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0039 - acc: 1.4926e-04
Epoch 300/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0036 - acc: 9.9507e-05
Epoch 301/200000
20099/20099 [==============================] - 0s 24us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 302/200000
20099/20099 [==============================] - 0s 22us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 303/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0043 - acc: 9.9507e-05
Epoch 304/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0034 - acc: 9.9507e-05
Epoch 305/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 9.

20099/20099 [==============================] - 0s 23us/step - loss: 0.0035 - acc: 9.9507e-05
Epoch 372/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 1.4926e-04
Epoch 373/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 9.9507e-05
Epoch 374/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 1.4926e-04
Epoch 375/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 1.4926e-04
Epoch 376/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0037 - acc: 1.4926e-04
Epoch 377/200000
20099/20099 [==============================] - 0s 23us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 378/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0034 - acc: 9.9507e-05
Epoch 379/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0036 - acc: 1.4926e-04
Epoch 380/200000
20099/2009

20099/20099 [==============================] - 0s 21us/step - loss: 0.0035 - acc: 1.4926e-04
Epoch 446/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0035 - acc: 9.9507e-05
Epoch 447/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0038 - acc: 9.9507e-05
Epoch 448/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0035 - acc: 9.9507e-05
Epoch 449/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0037 - acc: 9.9507e-05
Epoch 450/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0036 - acc: 1.4926e-04
Epoch 451/200000
20099/20099 [==============================] - 0s 20us/step - loss: 0.0035 - acc: 1.4926e-04
Epoch 452/200000
20099/20099 [==============================] - 0s 21us/step - loss: 0.0034 - acc: 9.9507e-05
Epoch 453/200000
20099/20099 [==============================] - 0s 24us/step - loss: 0.0037 - acc: 9.9507e-05
Epoch 454/200000
20099/2009

In [ ]:
res = model_last.predict([data_sky_test,data_t1_test,data_ndvi_test,data_ts_test,data_aod_test])
datares = res - data_pm_test
datares.PM25 = datares.PM25.map(lambda x: abs(x))
print(np.average(datares))

In [ ]:
data_predt = pd.concat([datares,data_pm_test], axis=1)
data_predt.columns = ["预测",'真']
data_predt['差值'] = (data_predt['预测']-data_predt["真"])
data_predt['差值'] = data_predt['差值'].map(lambda x: abs(x))
data_predt['百分误'] = data_predt['差值'].div(data_predt["真"])
data_predt.to_excel('baifenwu.xlsx')


inf
